In [24]:
import pandas as pd
import numpy as np
import re
import string
import random
from math import radians
import matplotlib.pyplot as plt

from gensim.models import KeyedVectors

from pythainlp.util import normalize, isthai
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp import word_vector

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import haversine_distances


### Data preprocessing

In [3]:
numberReview = 100

In [4]:
review = pd.read_csv("../data/file/review/review_500.csv")
review.head(5)

,placeName,review
0,เซ็นทรัลเวิลด์,ห้างใหญ่มาก อย่างที่กล่าวขานก็คือเดินจนหลง มาก...
1,เซ็นทรัลเวิลด์,ศูนย์การค้า Central wOrld 🛒🛍️ เป็นศูนย์การค้า ...
2,เซ็นทรัลเวิลด์,ช่วงเทศกาลเที่ยวกันอย่าประมาท หน้ากากอนามัยถอด...
3,เซ็นทรัลเวิลด์,เดินทางง่ายสะดวก ไม่ว่าจะเป็นรถส่วนตัว แท็กซี่...
4,เซ็นทรัลเวิลด์,สนุกกับช่วงเวลาแห่งการเฉลิมฉลองกับครอบครัวและค...


In [5]:
# Nan count data
sum(review['review'].isnull())

1480

In [6]:
review['review'] = review['review'].astype(str)

In [7]:
def isGoogleTranslateAndNaN(text: str):
    # Google Translae and nan value
    if (text.find("(แปลโดย Google)") > -1 or text=='nan' or text==""):
        return np.nan
    return text

def filterPlaceFromReviewNumber(review : pd.DataFrame, numberReview = 500):
    placeName = (review['placeName'].value_counts()[review['placeName'].value_counts() >= numberReview]).keys()
    
    review['placeName'] = review['placeName'].map(lambda x : x if x in placeName else np.nan)
    review.dropna(inplace=True)
    return review

def removeEmoji(string):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    return emoj.sub(r'', string)

def cleanText(text:str):
    """
    Normalize Function
    - Remove zero-width spaces
    - Remove duplicate spaces
    - Reorder tone marks and vowels to standard order/spelling
    - Remove duplicate vowels and signs
    - Remove duplicate tone marks
    - Remove dangling non-base characters at the beginning of text
    """
    text = removeEmoji(text)
    text = normalize(text)

    # remove <>, #, punctutaion in order
    text = re.sub(r'<.*?>','', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[a-z|A-Z|\d|(\a-z)]', '', text)
    text = re.sub(r'…', '', text)
    text = re.sub(r'ๆ', '', text)
    # URL pattern
    text = re.sub(r"^[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$", '', text)
    for c in string.punctuation:
        text = re.sub(r'\{}'.format(c), '', text)

    return text

def text_tokenizer(text:str):
    return [term for term in word_tokenize(text, engine='attacut') if isthai(term)]

def preprocessText(review: pd.DataFrame):
    review['review'] = review['review'].map(lambda x : isGoogleTranslateAndNaN(str(x)))
    review.dropna(inplace=True)
    review = filterPlaceFromReviewNumber(review, numberReview=numberReview)

    review['review'] = review['review'].map(lambda x : cleanText(x))

    return review

In [8]:
# Manage data
reviewClean = preprocessText(review=review)
print(f"Place left {len(reviewClean['placeName'].value_counts())} unit")
reviewClean

Place left 64 unit


,placeName,review
0,เซ็นทรัลเวิลด์,ห้างใหญ่มากอย่างที่กล่าวขานก็คือเดินจนหลงมากี่...
1,เซ็นทรัลเวิลด์,ศูนย์การค้า️เป็นศูนย์การค้าที่มีชื่อเดิมเรียกว...
2,เซ็นทรัลเวิลด์,ช่วงเทศกาลเที่ยวกันอย่าประมาทหน้ากากอนามัยถอดย...
3,เซ็นทรัลเวิลด์,เดินทางง่ายสะดวกไม่ว่าจะเป็นรถส่วนตัวแท็กซี่รถ...
4,เซ็นทรัลเวิลด์,สนุกกับช่วงเวลาแห่งการเฉลิมฉลองกับครอบครัวและค...
...,...,...
32598,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,เดินเพลิน
32599,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,สุดยอดครับ
32600,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,สะอาดสวยงาม
32601,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,มีที่จอดรถใช้ร่วมกับธนาคารเข้าไปจอดได้เลยค่าชม...


In [9]:
lengthDocument = 0
for r in reviewClean['review']:
    lengthDocument+=len(r)
lengthDocument

3372483

In [10]:
reviewClean["placeName"].value_counts()

เซ็นทรัลเวิลด์                                      500
วัดสุทัศนเทพวรารามราชวรมหาวิหาร                     500
สยามพารากอน                                         500
พิพิธภัณฑ์ช้างเอราวัณ                               500
คลองถมเซ็นเตอร์ (ตลาดคลองถม)                        500
                                                   ... 
แวร์เฮาส์ 30                                        180
มาดามทุสโซ                                          170
พิพิธภัณฑสถานแห่งชาติ หอศิลป์                       160
พิพิธภัณฑ์ชาวบางกอก                                 152
หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ    103
Name: placeName, Length: 64, dtype: int64

### TFIDF

In [ ]:
stop_words = [t.encode('utf-8') for t in list(thai_stopwords())]

tfidf_vector = TfidfVectorizer(
    tokenizer=text_tokenizer,
    ngram_range=(2,3),
    stop_words=stop_words,
    max_features=5000
)

In [ ]:
X = tfidf_vector.fit_transform(reviewClean['review'])

In [ ]:
tfidf_vector.get_feature_names_out()

array(['กระเป๋า รองเท้า', 'กราบ ขอ', 'กราบ ขอ พร', ..., 'ไหว้ พระ ขอ',
       'ไหว้ ศาล', 'ไหว้ ศาล หลักเมือง'], dtype=object)

In [ ]:
term_tfidf = pd.DataFrame(
    X.toarray(),
    columns=tfidf_vector.get_feature_names(),
    index=reviewClean.index
)

c:\Users\Pongs\miniconda3\envs\TripRecommendation\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
term_tfidf

,กก กก,กก กก กก,กก มี,กระเป๋า รองเท้า,กราบ ขอพร,กราบ สักการะ,กราบ หลวงพ่อ,กราบ ไหว้พระ,กราบพระ แก้วมรกต,กราบไหว้ ขอพร,...,ไหว้ หลวงพ่อ,ไหว้ แก้,ไหว้ ได้,ไหว้พระ ขอพร,ไหว้พระ ทำบุญ,ไหว้พระ วัด,ไหว้พระ แก้วมรกต,ไหว้พระ และ,ไห้ ว,ไอคอน สยาม
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
term_tfidf.sum().sort_values(ascending=False).head(20)

จอด รถ           472.674563
ที่ จอด          407.489362
ที่ จอด รถ       360.100845
ดี มาก           354.514714
เยอะ มาก         337.187414
บรรยากาศ ดี      269.788406
ของ กิน          262.318243
ที่ นี่          244.566984
ร้าน อาหาร       242.650551
ถ่าย รูป         218.500375
ออก กำลัง        217.571780
มาก ค่ะ          212.315078
สวย มาก          211.112160
มาก ครับ         210.148497
ร้าน ค้า         208.865586
มี ร้าน          205.964330
ให้ เลือก        203.670750
กำลัง กาย        201.463740
ออก กำลัง กาย    200.686109
สวยงาม มาก       192.098238
dtype: float64

In [38]:
numberReview_dict = {}
for idx, name in enumerate(list(reviewClean['placeName'].value_counts().keys())):
    numberReview_dict[name] = reviewClean['placeName'].value_counts()[idx]

In [39]:
len(idx_type_vector['art']), len(idx_type_vector['history']), len(idx_type_vector['nature']), len(idx_type_vector['shopping'])

(20, 20, 20, 20)

In [40]:
# Combination every review to 1 vector each place

numberPlace = len(reviewClean['placeName'].value_counts())
idx = 0
tfidfList = []
start = 0

for idx, name in enumerate(reviewClean['placeName'].unique()):
    end = start + numberReview_dict[name]
    # print(start, end, numberReview_dict[name])
    i_tfidf = term_tfidf.iloc[start:end].sum() / numberReview_dict[name]

    tfidfList.append(i_tfidf.to_numpy())

    start += numberReview_dict[name]

NameError: name 'term_tfidf' is not defined

In [41]:
tfidf_vector_dict = {}

for idx, place in enumerate(reviewClean['placeName'].unique()):
     tfidf_vector_dict[place] = tfidfList[idx]

IndexError: list index out of range

In [42]:
detail = pd.read_csv('../data/file/detailPlace.csv')
detail['tfidf'] = np.NaN
detail['tfidf'] = detail['tfidf'].astype(object)
len(detail)

75

In [28]:
for idx in range(len(tfidf_vector_dict)):
    try:
        detail_place = detail.iloc[idx, 1]
        # print(detail_place)
        value = tfidf_vector_dict[detail_place]
        detail.iloc[idx, -1] = value.reshape(-1,1)
    except Exception as error:
        print(error)
        continue

NameError: name 'tfidf_vector_dict' is not defined

In [ ]:
detail.dropna(subset=['tfidf'],inplace=True)
detail = detail.reset_index(drop=True)
detail['no'] = [i+1 for i in range(len(detail))]

In [ ]:
detail.insert(2, 'type2', webDetail['type'])
detail.insert(2, 'placeID', webDetail['placeId'])

In [ ]:
detail.to_csv('detailTFIDF.csv', index=False)

In [10]:
webDetail = pd.read_csv('../data/file/finalPlace.csv')
webDetail['use'] = np.NaN
placeName = list(detail['name'].unique())

for idx, place in enumerate(webDetail['placeName']):
    for p in placeName:
        if place==p:
            webDetail.loc[idx, "use"] = 1

webDetail.dropna(inplace=True)
webDetail = webDetail.reset_index(drop=True)

NameError: name 'detail' is not defined

In [44]:
# Vector ['art', 'history', 'nature', 'shopping', 'special']
type_list = ['art', 'history', 'nature', 'shopping', 'special']
type_vector = np.zeros((len(webDetail), len(type_list)))

for idx in range(len(webDetail)):
    type_vector_i = np.zeros(5)
    for t in webDetail['type'][idx].split(','):
        for i, t_i in enumerate(type_list):
            if t==t_i:
                type_vector_i[i]=1
    
    type_vector[idx] = type_vector_i

idx_type_vector = {
    "art":[],
    "history":[],
    "nature":[],
    "shopping":[],
    "special":[]
}

for idx, type_vector_i in enumerate(type_vector):
    for i, x in enumerate(type_vector_i):
        if x==1:
            if i==0:
                idx_type_vector["art"].append(idx)
            if i==1:
                idx_type_vector["history"].append(idx)
            if i==2:
                idx_type_vector["nature"].append(idx)
            if i==3:
                idx_type_vector["shopping"].append(idx)
            if i==4:
                idx_type_vector["special"].append(idx)

len(idx_type_vector['art']), len(idx_type_vector['history']), len(idx_type_vector['nature']), len(idx_type_vector['shopping'])

(20, 20, 20, 20)

### word2vec

In [26]:
#load into gensim
model = KeyedVectors.load_word2vec_format("LTW2V_v1.0-window5.bin",binary=True, unicode_errors='ignore')
# model = word_vector.WordVector(model_name="thai2fit_wv").get_model() # load thai2fit_wv from pythainlp

In [47]:
model['แมว'].shape

(400,)

In [48]:
len(model.index_to_key)

596005

In [49]:
reviewClean_tokenized = pd.read_csv("reviewTokenized.csv")
reviewClean_tokenized

,placeName,review,tokenized
0,เซ็นทรัลเวิลด์,ห้างใหญ่มากอย่างที่กล่าวขานก็คือเดินจนหลงมากี่...,"['ห้าง', 'ใหญ่', 'มาก', 'อย่าง', 'ที่', 'กล่าว..."
1,เซ็นทรัลเวิลด์,ศูนย์การค้า️เป็นศูนย์การค้าที่มีชื่อเดิมเรียกว...,"['ศูนย์การค้า', '️', 'เป็น', 'ศูนย์', 'การ', '..."
2,เซ็นทรัลเวิลด์,ช่วงเทศกาลเที่ยวกันอย่าประมาทหน้ากากอนามัยถอดย...,"['ช่วง', 'เทศกาล', 'เที่ยว', 'กัน', 'อย่า', 'ป..."
3,เซ็นทรัลเวิลด์,เดินทางง่ายสะดวกไม่ว่าจะเป็นรถส่วนตัวแท็กซี่รถ...,"['เดินทาง', 'ง่าย', 'สะดวก', 'ไม่ว่า', 'จะ', '..."
4,เซ็นทรัลเวิลด์,สนุกกับช่วงเวลาแห่งการเฉลิมฉลองกับครอบครัวและค...,"['สนุก', 'กับ', 'ช่วง', 'เวลา', 'แห่ง', 'การ',..."
...,...,...,...
29594,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,เดินเพลิน,"['เดิน', 'เพลิน']"
29595,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,สุดยอดครับ,"['สุด', 'ยอด', 'ครับ']"
29596,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,สะอาดสวยงาม,"['สะอาด', 'สวยงาม']"
29597,หอศิลป์สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินีนาถ,มีที่จอดรถใช้ร่วมกับธนาคารเข้าไปจอดได้เลยค่าชม...,"['มี', 'ที่', 'จอด', 'รถ', 'ใช้', 'ร่วม', 'กับ..."


In [50]:
lengthDocument = 0
for t in reviewClean_tokenized['tokenized']:
    lengthDocument+=len(t)
lengthDocument

6752626

In [51]:
len(reviewClean_tokenized['placeName'].unique())

64

In [52]:
review_vector_list = []
for word_tokenized in reviewClean_tokenized['tokenized']:
    word_vector = np.zeros(model['คำ'].shape)
    for word in word_tokenized:
        if word in model.key_to_index:
            word_vector += model[word]
    review_vector_list.append(word_vector)

In [53]:
review_number_dict = {}

for name, number in reviewClean_tokenized['placeName'].value_counts().items():
    review_number_dict[name] = number

In [54]:
idx = 0
term_vector_word2vec = {}
start = 0

for idx, name in enumerate(webDetail['placeName'].unique()):
    end = start + review_number_dict[name]
    # print(start, end, numberReview_dict[name])
    vector_i = sum(review_vector_list[start:end]) / review_number_dict[name]

    term_vector_word2vec[name] = vector_i

    start += review_number_dict[name]

In [55]:
len(term_vector_word2vec)

62

### TFIDF score & Recommend from term score

In [27]:
detail = pd.read_csv("detailTFIDF.csv")
detail.head(3)

,no,name,placeID,type2,detail,business_status,address,geometry,phone_number,url,type,period,tfidf
0,1,เซ็นทรัลเวิลด์,ChIJ4VX0ws-e4jARBGaQ2IACrcQ,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,"4, 5 ถนน ราชดำริ แขวงปทุมวัน เขตปทุมวัน กรุงเท...","(13.7460002, 100.5399162)",+66 2 640 7000,https://maps.google.com/?cid=14171986354817230340,"shopping_mall,point_of_interest,establishment","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.000664085114051197..."
1,2,สยามพารากอน,ChIJIeWu482e4jARYymvLJqTQ58,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,991 ถ. พระรามที่ ๑ แขวงปทุมวัน เขตปทุมวัน กรุง...,"(13.7462411, 100.5347402)",+66 2 690 1000,https://maps.google.com/?cid=11476178565854079331,"shopping_mall,department_store,store,point_of_...","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.00028894443008002245, 0.0, 0.0, 0.0, 0.0, 0..."
2,3,พระบรมมหาราชวัง,ChIJPzZsMU6Z4jARQUzvk913bCo,"history,nature","{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,ถนน หน้าพระลาน แขวงพระบรมมหาราชวัง เขตพระนคร ก...,"(13.7498558, 100.4915765)",-,https://maps.google.com/?cid=3056950040631135297,"tourist_attraction,park,point_of_interest,esta...","{'monday': '8:30–15:30', 'tuesday': '8:30–15:3...","[0.0, 0.0003690889367503711, 0.000384252038422..."


In [28]:
print(detail['tfidf'].to_numpy().shape)
print(np.array([np.fromstring(detail['tfidf'].to_numpy()[0][1:-1], sep=',')]).shape)

(62,)
(1, 5000)


In [29]:
# matrix of tfidf score
def get_termVector(detail: pd.DataFrame):
    term_vector = np.array([np.fromstring(detail['tfidf'].to_numpy()[0][1:-1], sep=',')])

    for i in range(len(detail) - 1):
        term_vector_i = np.array([np.fromstring(detail['tfidf'].to_numpy()[i+1][1:-1], sep=',')])
        term_vector = np.append(term_vector, term_vector_i, axis=0)
    
    return term_vector

In [30]:
term_vector = get_termVector(detail)
term_vector.shape

(62, 5000)

In [31]:
term_score = cosine_similarity(term_vector)
term_score.shape

(62, 62)

In [32]:
def recommend(place):
    index = detail[detail['name'] == place]['no'].values[0] - 1
    distances = sorted(list(enumerate(term_score[index].reshape(-1))), reverse=True, key=lambda x:x[1])
    for i in distances[1:11]:
        print(detail.iloc[i[0]]['name'])

In [33]:
recommend('วัดอรุณราชวรารามราชวรมหาวิหาร (วัดแจ้ง)')

วัดพระศรีรัตนศาสดาราม (วัดพระแก้ว)
วัดสุทัศนเทพวรารามราชวรมหาวิหาร
พระบรมมหาราชวัง
วัดนาคปรก
วัดไตรมิตรวิทยารามวรวิหาร
วัดประยุรวงศาวาสวรวิหาร
วัดสระเกศราชวรมหาวิหาร (ภูเขาทอง)
วัดธรรมมงคลเถาบุญนนทวิหาร(หลวงพ่อวิริยังค์)
สนามหลวง
เอเชียทีค เดอะ ริเวอร์ฟรอนท์


## Full Recommendation system
- [x] TermScore
- [x] RatingScore
- [x] ReviewScore
- [x] TypeScore

In [34]:
detail = pd.read_csv("detailTFIDF.csv")
detail.head(3)

,no,name,placeID,type2,detail,business_status,address,geometry,phone_number,url,type,period,tfidf
0,1,เซ็นทรัลเวิลด์,ChIJ4VX0ws-e4jARBGaQ2IACrcQ,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,"4, 5 ถนน ราชดำริ แขวงปทุมวัน เขตปทุมวัน กรุงเท...","(13.7460002, 100.5399162)",+66 2 640 7000,https://maps.google.com/?cid=14171986354817230340,"shopping_mall,point_of_interest,establishment","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.000664085114051197..."
1,2,สยามพารากอน,ChIJIeWu482e4jARYymvLJqTQ58,shopping,"{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,991 ถ. พระรามที่ ๑ แขวงปทุมวัน เขตปทุมวัน กรุง...,"(13.7462411, 100.5347402)",+66 2 690 1000,https://maps.google.com/?cid=11476178565854079331,"shopping_mall,department_store,store,point_of_...","{'monday': '10:00–22:00', 'tuesday': '10:00–22...","[0.00028894443008002245, 0.0, 0.0, 0.0, 0.0, 0..."
2,3,พระบรมมหาราชวัง,ChIJPzZsMU6Z4jARQUzvk913bCo,"history,nature","{'business_status': 'OPERATIONAL', 'formatted_...",OPERATIONAL,ถนน หน้าพระลาน แขวงพระบรมมหาราชวัง เขตพระนคร ก...,"(13.7498558, 100.4915765)",-,https://maps.google.com/?cid=3056950040631135297,"tourist_attraction,park,point_of_interest,esta...","{'monday': '8:30–15:30', 'tuesday': '8:30–15:3...","[0.0, 0.0003690889367503711, 0.000384252038422..."


In [35]:
webDetail = pd.read_csv('../data/file/finalPlace.csv')
webDetail['use'] = np.NaN
placeName = list(detail['name'].unique())

for idx, place in enumerate(webDetail['placeName']):
    for p in placeName:
        if place==p:
            webDetail.loc[idx, "use"] = 1

webDetail.dropna(inplace=True)
webDetail = webDetail.reset_index(drop=True)

In [36]:
len(webDetail)

62

In [37]:
# Type from Place API (Too much type)

type_set = set()

for idx, type_i in enumerate(detail['type2']):
    for t in type_i.split(','):
        type_set.add(t)

type_list = sorted(list(type_set))
type_list

['art', 'history', 'nature', 'shopping', 'special']

In [38]:
# Vector ['art', 'history', 'nature', 'shopping', 'special']
type_vector = np.zeros((len(detail), len(type_list)))

for idx in range(len(detail)):
    type_vector_i = np.zeros(5)
    for t in detail['type2'][idx].split(','):
        for i, t_i in enumerate(type_list):
            if t==t_i:
                type_vector_i[i]=1
    
    type_vector[idx] = type_vector_i

In [114]:
# load Term vector word2vec
import pickle
with open('term_word2vec.pkl', 'rb') as file:
    term_vector_word2vec = pickle.load(file)
term_vector_word2vec 

{'เซ็นทรัลเวิลด์': array([-183.88608044,  -60.75601703,  147.88392068, -295.77904183,
         349.49069679, -251.13982719,  -31.7974232 , -206.82168895,
        -114.14896617,  175.24506568,    3.36263485,    5.98772068,
         512.49368189,  121.78998802, -688.79821101,  508.636533  ,
         182.49337029,  169.03383755, -209.4267371 ,  422.29587939,
         196.86211339,   78.04179258,   62.6513515 , -111.9268168 ,
        -178.28705539, -165.07662742,  265.36209363, -174.07085521,
        -386.76943975,  124.04746527, -240.39768665,  -44.44112605,
         103.17695296,   81.20572874,  515.15256397, -220.41168689,
        -177.59913096, -518.22274436, -191.91536006,  -74.85418877,
         336.65734104,  198.66926667, -113.45703681, -167.53515895,
         -18.56794197,   84.37451222,  165.42277365,  -30.64593785,
        -128.65413566,   49.3063063 ,  158.34434515,  -34.31503762,
        -167.31412206,  116.06349768,  235.37461512,  230.5361971 ,
        -296.20172421,  104.26

In [115]:
def get_random_index(trip_level): # Level 0,1,2
    # Calcuate ratio of each type place
    sum_level = float(sum(trip_level))
    percentByLevel = 1. / sum_level 
    ratioLevel = [level*percentByLevel for level in trip_level]
    print(ratioLevel)

    # Find index of each type from type vector (Adding more History before TFIDF)
    idx_type_vector = {
        "art":[],
        "history":[],
        "nature":[],
        "shopping":[],
        # "special":[]
    }

    for idx, type_vector_i in enumerate(type_vector):
        for i, x in enumerate(type_vector_i):
            if x==1:
                if i==0:
                    idx_type_vector["art"].append(idx)
                if i==1:
                    idx_type_vector["history"].append(idx)
                if i==2:
                    idx_type_vector["nature"].append(idx)
                if i==3:
                    idx_type_vector["shopping"].append(idx)
                # if i==4:
                #     idx_type_vector["special"].append(idx)

    print(len(idx_type_vector['art']), len(idx_type_vector['history']), len(idx_type_vector['nature']), len(idx_type_vector['shopping']))

    # Random choices place index follow Trip level
    all_random_index = set()
    random_index = {}

    for idx, typeName in enumerate(idx_type_vector):
        if ratioLevel[idx] > 0:
            random_index[typeName] = random.sample(idx_type_vector[typeName], k=int(ratioLevel[idx]*len(idx_type_vector[typeName]))) 

    # Input in set don't want a same index

    for idx, typeName in enumerate(idx_type_vector):
        
        if ratioLevel[idx] > 0:
            for i in random_index[typeName]: all_random_index.add(i)

    all_random_index = list(all_random_index)
    
    return ratioLevel, all_random_index


def get_term_score(all_random_index, model='tfidf'):
    if model=='tfidf':
        # Term vector
        # Select vector from upper index
        trip_term_vector = np.zeros(term_vector[0].shape)

        for idx_vector in all_random_index:
            trip_term_vector += term_vector[idx_vector]

        trip_term_vector = trip_term_vector / len(all_random_index)
        trip_term_score = cosine_similarity(np.append(term_vector, trip_term_vector.reshape(1,-1), axis=0))
        return trip_term_score
    if model=='word2vec':
        # Term vector
        # Select vector from upper index
        trip_term_vector = np.zeros(term_vector_word2vec["เซ็นทรัลเวิลด์"].shape)

        for idx_vector in all_random_index:
            trip_term_vector += term_vector_word2vec[detail.iloc[idx_vector, 1]]

        trip_term_vector = trip_term_vector / len(all_random_index)
        trip_term_score = cosine_similarity(np.append(np.array(list(term_vector_word2vec.values())), trip_term_vector.reshape(1,-1), axis=0))
        return trip_term_score
        

def get_type_score(all_random_index):
    # Type vector
    trip_type_vector = np.zeros(type_vector[0].shape)

    for idx_vector in all_random_index:
        trip_type_vector += type_vector[idx_vector]

    trip_type_vector = trip_type_vector / len(all_random_index)
    trip_type_score = cosine_similarity(np.append(type_vector, trip_type_vector.reshape(1,-1), axis=0))
    return trip_type_score

# Distance (lat,long)
def get_distance_score(place_index):
    start_idx = place_index

    distance = np.zeros((62,))
    geometry_list = detail['geometry'].map(lambda x  : eval(x)).to_list()
    
    current_point = eval(detail.iloc[start_idx]['geometry'])
    visited[start_idx] = 1

    current_radian = [radians(_) for _ in current_point]

    for idx, dest_point in enumerate(geometry_list):
        dest_radian = [radians(_) for _ in dest_point]
        distance[idx] = haversine_distances([current_radian, dest_radian])[0][1] * 6371 # 6371 is radius of the Earth
    
    return np.append(1/distance, np.Inf)

def visualize_map(track_geometry, track_name):
    import plotly.graph_objects as go

    fig = go.Figure(data=go.Scattergeo(
            lon = detail['geometry'].map(lambda x : eval(x)[1]),
            lat = detail['geometry'].map(lambda x : eval(x)[0]),
            text = detail['name'],
            mode = 'markers',
            marker_color = visited[:62]
            ))

    fig.add_trace(go.Scattergeo(
        mode = "lines",
        lon = list(map(lambda x : x[1], track_geometry)),
        lat = list(map(lambda x : x[0], track_geometry)),
        text = track_name,
        textposition = "top center",
        line = dict(width = 1, color = "red"),
    ))

    fig.update_layout(
            title = 'Bangkok Recommendation system',
            geo_scope='asia',
        )
    fig.show()

TFIDF recommend

In [116]:
# Trip vector level 1-5 ['art', 'history', 'nature', 'special', 'shopping']
# This input emphasize the history place
# select level unit (mulitpy 1)
trip_level = [2,0,0,2] # Level ['art', 'history', 'nature', 'shopping']
visited = np.zeros((len(detail)+1,))
visited[62] = 1
track_geometry = []
track_name = []

ratioLevel, random_index = get_random_index(trip_level)
trip_term_score = get_term_score(random_index, model='tfidf')
trip_type_score = get_type_score(random_index)

# Rating score
rating_score = webDetail['rating'].to_numpy()
rating_score = np.append(rating_score, 5)
# Distance score
trip_distance_score = np.zeros((len(detail),))
trip_distance_score = np.append(trip_distance_score, np.Inf)

for i in range(10):
    # tfidf term_score is important score because the different of value are far
    relevance_score = trip_term_score[-1]*2 + trip_type_score[-1]*1 + rating_score*0.05 + trip_distance_score * 1

    # Recommendation
    place_recommended = sorted(list(enumerate(relevance_score.reshape(-1))), reverse=True, key=lambda x:x[1])
    for idx, similar in place_recommended:
        # print(idx)
        if not visited[idx]:
            print(detail.iloc[idx]['name'], detail.iloc[idx]['type2'])
            track_geometry.append(eval(detail.iloc[idx]['geometry']))
            track_name.append(f"{i+1} {detail.iloc[idx]['name']}")
            trip_distance_score = get_distance_score(idx)
            visited[idx]=1
            break
visualize_map(track_geometry, track_name)

[0.5, 0.0, 0.0, 0.5]
9 20 20 20
เอ็ม บี เค เซ็นเตอร์ shopping
หอศิลปวัฒนธรรมแห่งกรุงเทพมหานคร history,art,special
มาดามทุสโซ history
สยามพารากอน shopping
ซีไลฟ์ แบงคอก nature,special
วัดปทุมวนาราม ราชวรวิหาร history,nature
เซ็นทรัลเวิลด์ shopping
เดอะ แพลทินัม แฟชั่น มอลล์ shopping
เซ็นทรัล ชิดลม shopping
เทอร์มินอล 21 อโศก shopping


C:\Users\Pongs\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:105: RuntimeWarning:

divide by zero encountered in true_divide



In [117]:
detail.iloc[random_index, 1].to_list()

['เซ็นทรัลเวิลด์',
 'ตลาดสำเพ็ง',
 'คลองถมเซ็นเตอร์ (ตลาดคลองถม)',
 'สยามพารากอน',
 'ตลาดนัดจตุจักร',
 'ตลาดน้ำคลองลัดมะยม',
 'เทอร์มินอล 21 อโศก',
 'เมก้า พลาซ่า สะพานเหล็ก',
 'พิพิธภัณฑ์ศิลปะไทยร่วมสมัย',
 'เอ็ม บี เค เซ็นเตอร์',
 'เซ็นทรัล พระราม 2',
 'นิทรรศน์รัตนโกสินทร์',
 'พิพิธภัณฑ์ชาวบางกอก',
 'แวร์เฮาส์ 30']

### word2vec recommend

In [119]:
# Trip vector level 1-5 ['art', 'history', 'nature', 'special', 'shopping']
# This input emphasize the history place
# select level unit (mulitpy 1)
trip_level = [2,0,0,2] # Level ['art', 'history', 'nature', 'shopping']
visited = np.zeros((len(detail)+1,))
visited[62] = 1

tracking = {
    "idx":[],
    "name":[],
    "type":[],
    "geometry":[],
}
track_geometry = []
track_name = []

ratioLevel, random_index = get_random_index(trip_level)
trip_term_score = get_term_score(random_index, model='word2vec')
trip_type_score = get_type_score(random_index)

# Rating score
rating_score = webDetail['rating'].to_numpy()
rating_score = np.append(rating_score, 5)
# Distance score
trip_distance_score = np.zeros((len(detail),))
trip_distance_score = np.append(trip_distance_score, np.Inf)

for i in range(10):
    # word2vec term_score is very close so adjust other part is a good idea!
    relevance_score = trip_term_score[-1]*1 + trip_type_score[-1]*1 + rating_score*0.05 + trip_distance_score * 1

    # Recommendation
    place_recommended = sorted(list(enumerate(relevance_score.reshape(-1))), reverse=True, key=lambda x:x[1])
    for idx, similar in place_recommended:
        # print(idx)
        if not visited[idx]:
            print(detail.iloc[idx]['name'], detail.iloc[idx]['type2'])
            trip_distance_score = get_distance_score(idx)
            visited[idx]=1
            tracking['geometry'].append(eval(detail.iloc[idx]['geometry']))
            tracking['name'].append(detail.iloc[idx]['name'])
            tracking['idx'].append(idx)
            tracking['type'].append(detail.iloc[idx]["type2"])
            break
visualize_map(tracking['geometry'], tracking['name'])

[0.5, 0.0, 0.0, 0.5]
9 20 20 20
เซ็นทรัลเวิลด์ shopping
วัดปทุมวนาราม ราชวรวิหาร history,nature
ซีไลฟ์ แบงคอก nature,special
สยามพารากอน shopping
มาดามทุสโซ history
หอศิลปวัฒนธรรมแห่งกรุงเทพมหานคร history,art,special
เอ็ม บี เค เซ็นเตอร์ shopping
เดอะ แพลทินัม แฟชั่น มอลล์ shopping
เซ็นทรัล ชิดลม shopping
เทอร์มินอล 21 อโศก shopping


C:\Users\Pongs\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:105: RuntimeWarning:

divide by zero encountered in true_divide



In [120]:
detail.iloc[random_index, 1].to_list()

['เซ็นทรัลเวิลด์',
 'สยามพารากอน',
 'คลองถมเซ็นเตอร์ (ตลาดคลองถม)',
 'ตลาดนัดเลียบด่วน รามอินทรา',
 'ตลาดน้ำคลองลัดมะยม',
 'เมก้า พลาซ่า สะพานเหล็ก',
 'ปากคลองตลาด',
 'มิวเซียมสยาม',
 'เซ็นทรัล พระราม 9',
 'หอศิลปวัฒนธรรมแห่งกรุงเทพมหานคร',
 'ริเวอร์ซิตี้ แบงค็อก',
 'นิทรรศน์รัตนโกสินทร์',
 'ตลาดนกฮูก',
 'หอศิลป์ร่วมสมัยราชดำเนิน']

In [121]:
import datetime
import time

placePerDay = 3
duration = 6. #Hour
day = 3 #day

dt = datetime.datetime.strptime(f'03/16/23 11:00', '%m/%d/%y %H:%M')
millisenconds = int(time.mktime(dt.timetuple())) * 1000 # input from web
time_start = datetime.datetime.fromtimestamp(millisenconds/1000)

HourPerPlace = int(duration / placePerDay)
MinPerPlace = ((duration/placePerDay)*60) - (HourPerPlace*60)

planning = {}

idx = 0

for d in range(day):
    placeIn = []
    time_start = datetime.datetime.fromtimestamp(millisenconds/1000) + datetime.timedelta(days=d+1)
    for j in range(placePerDay):
        time_end = time_start + datetime.timedelta(hours=HourPerPlace, minutes=MinPerPlace)
        milli_time_start = int(time.mktime(time_start.timetuple())) * 1000
        milli_time_end = int(time.mktime(time_end.timetuple())) * 1000

        placeIn.append({
            'name': tracking['name'][idx],
            'geometry': { "lat" : detail.iloc[tracking['idx'][idx]]['geometry'].split(',')[0][1:],
                            "lon": detail.iloc[tracking['idx'][idx]]['geometry'].split(',')[-1][:-1]
                        },
            'time': {"time_start": time_start.strftime("%Y-%m-%d %H:%M"),
                     "time_end" :  time_end.strftime("%Y-%m-%d %H:%M")
                     }
        })
        time_start = time_end
        idx+=1
    planning[str(d+1)] = placeIn

# time_end = time_start + datetime.timedelta(hours=timePerPlace)
# time_end.strftime("%Y-%m-%d %H:%M")

In [122]:
planning

{'1': [{'name': 'เซ็นทรัลเวิลด์',
   'geometry': {'lat': '13.7460002', 'lon': ' 100.5399162'},
   'time': {'time_start': '2023-03-17 11:00', 'time_end': '2023-03-17 13:00'}},
  {'name': 'วัดปทุมวนาราม ราชวรวิหาร',
   'geometry': {'lat': '13.7460018', 'lon': ' 100.5370069'},
   'time': {'time_start': '2023-03-17 13:00', 'time_end': '2023-03-17 15:00'}},
  {'name': 'ซีไลฟ์ แบงคอก',
   'geometry': {'lat': '13.7459351', 'lon': ' 100.5352057'},
   'time': {'time_start': '2023-03-17 15:00',
    'time_end': '2023-03-17 17:00'}}],
 '2': [{'name': 'สยามพารากอน',
   'geometry': {'lat': '13.7462411', 'lon': ' 100.5347402'},
   'time': {'time_start': '2023-03-18 11:00', 'time_end': '2023-03-18 13:00'}},
  {'name': 'มาดามทุสโซ',
   'geometry': {'lat': '13.746418', 'lon': ' 100.531671'},
   'time': {'time_start': '2023-03-18 13:00', 'time_end': '2023-03-18 15:00'}},
  {'name': 'หอศิลปวัฒนธรรมแห่งกรุงเทพมหานคร',
   'geometry': {'lat': '13.7466997', 'lon': ' 100.5303705'},
   'time': {'time_start': '2